# Week 3 Lab - Basics of TensorFlow and PyTorch

## Verifying GPU Availability

### TensorFlow GPU Check

In [1]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("GPU is available for TensorFlow!")
else:
    print("No GPU found for TensorFlow.")


TensorFlow version: 2.15.0
GPU is available for TensorFlow!


### PyTorch GPU Check

In [ ]:
import torch

print("PyTorch version:", torch.__version__)
if torch.cuda.is_available():
    print("GPU is available for PyTorch!")
else:
    print("No GPU found for PyTorch.")


PyTorch version: 2.3.0
GPU is available for PyTorch!


## Creating and Training a Simple Neural Network

### TensorFlow Implementation

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Load and preprocess the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Define the model
model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test accuracy: {accuracy:.4f}')


11490434/11490434 [==============================] - 2s 0us/step
Epoch 1/5
1875/1875 [==============================] - 9s 4ms/step - loss: 0.2539 - accuracy: 0.9276 - val_loss: 0.1372 - val_accuracy: 0.9607
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.1136 - accuracy: 0.9663 - val_loss: 0.0943 - val_accuracy: 0.9711
Epoch 3/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.0791 - accuracy: 0.9761 - val_loss: 0.0844 - val_accuracy: 0.9721
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.0604 - accuracy: 0.9815 - val_loss: 0.0790 - val_accuracy: 0.9755
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 0.0837 - accuracy: 0.9735
Test accuracy: 0.9735


### PyTorch Implementation

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Load and preprocess the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

# Define the model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(28*28, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleNN()

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')


Epoch [1/5], Loss: 0.1931
Epoch [2/5], Loss: 0.0363
Epoch [3/5], Loss: 0.3085
Epoch [4/5], Loss: 0.1344
Epoch [5/5], Loss: 0.0438
Test Accuracy: 96.21%


---
### Questions
Answer the following questions in detail.

1. What is the purpose of normalizing the input data in both TensorFlow and PyTorch implementations?
2. Explain the role of the activation function relu in the neural network.
3. Why is it important to use GPU for training neural networks?
4. Compare the training time and accuracy of the TensorFlow and PyTorch models. Which one performed better and why?


---
### Answer

1. to prevent imbalance feature that would affect gradient during backpropagation in the training. as well as improve training time, by normalizing data
some data that have very different range ie house price(range 500,000-1,000,000+) with area (500-1000) could take more time for the model to converge 
2. to introduce non-linearity to the model, without it the model will be single linear transformation (unable to pickup complex pattern)
and being efficient - if the input is <0, the output is 0, if the input is >0, output remain the same
3. Faster, GPU provide more processing power than CPU in both memory bandwidth and paralel processing that able to run multiple calculation at the same time (also seen usage in bitcoin mining, similar reason plus scalability that multiple GPU can be chain up together for evenmore computational power) on top of that is Manufacturer also provide optimized library for neural network/machine learning namely NVIDIA's cuDNN (CUDA Deep Neural Network library)
4. Tensorflow (56 sec) is  faster than pytorch (1m 38 sec), but most of the setting is the same
    - using same dataset (MNIST), same network structure (28*28 input, to 128 nodes in 2nd layer and 10 node in output layers)
    - same optimizer as well being Adam and learning 0.001, (from quick check default learning rate is 0.001, so even it not set, both still using same value )
    - same epoch count, same loss function (cross entropy)
    so I think that it come down to be a example on tensorflow and pytorch performance, the library itself. that tensorflow is generally more efficient/powerful than pytorch
